## Deliverable 2. Create a Customer Travel Destinations Map.

In [1]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [2]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("Weather_Database/WeatherPy_database.csv")
city_data_df.head()

,City_ID,City,Country,Date,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Tuktoyaktuk,CA,2022-02-11 07:21:04,69.4541,-133.0374,-16.56,94,100,23.31,overcast clouds
1,1,Hobart,AU,2022-02-11 07:20:19,-42.8794,147.3294,69.03,52,20,19.57,few clouds
2,2,Tuatapere,NZ,2022-02-11 07:21:05,-46.1333,167.6833,59.83,80,100,8.70,overcast clouds
3,3,Ponta Do Sol,PT,2022-02-11 07:21:05,32.6667,-17.1000,65.71,61,100,7.58,overcast clouds
4,4,Baykit,RU,2022-02-11 07:21:06,61.6700,96.3700,4.44,82,6,1.14,clear sky


In [3]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 75
What is the maximum temperature you would like for your trip? 90


In [4]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head()

,City_ID,City,Country,Date,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
18,18,Saint-Philippe,RE,2022-02-11 07:21:10,-21.3585,55.7679,83.75,76,58,8.37,broken clouds
21,21,Vondrozo,MG,2022-02-11 07:21:11,-22.8167,47.2833,79.65,64,70,3.87,broken clouds
22,22,Vaini,TO,2022-02-11 07:18:45,-21.2000,-175.2000,75.36,100,75,11.50,moderate rain
28,28,Tabou,CI,2022-02-11 07:21:14,4.4230,-7.3528,78.85,87,72,5.19,light rain
32,32,Rikitea,PF,2022-02-11 07:21:15,-23.1203,-134.9692,79.34,74,14,17.67,few clouds


In [5]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.notnull().sum()

City_ID                160
City                   160
Country                156
Date                   160
Lat                    160
Lng                    160
Max Temp               160
Humidity               160
Cloudiness             160
Wind Speed             160
Current Description    160
dtype: int64

In [6]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
preferred_cities_df.dropna()
preferred_cities_clean_df = preferred_cities_df

In [7]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = preferred_cities_clean_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
18,Saint-Philippe,RE,83.75,broken clouds,-21.3585,55.7679,
21,Vondrozo,MG,79.65,broken clouds,-22.8167,47.2833,
22,Vaini,TO,75.36,moderate rain,-21.2000,-175.2000,
28,Tabou,CI,78.85,light rain,4.4230,-7.3528,
32,Rikitea,PF,79.34,few clouds,-23.1203,-134.9692,
35,Hithadhoo,MV,81.30,overcast clouds,-0.6000,73.0833,
38,Perbaungan,ID,86.11,overcast clouds,3.5679,98.9560,
40,Kavaratti,IN,80.47,overcast clouds,10.5669,72.6420,
44,Quatre Cocos,MU,84.31,scattered clouds,-20.2078,57.7625,
53,Isangel,VU,80.53,overcast clouds,-19.5500,169.2667,


In [9]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():

    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    
    # Add the latitude and longitude to location key for the params dictionary 
    params["location"] = f"{lat},{lng}"
    
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found...skipping")   
        

Hotel not found...skipping
Hotel not found...skipping
Hotel not found...skipping
Hotel not found...skipping
Hotel not found...skipping
Hotel not found...skipping
Hotel not found...skipping
Hotel not found...skipping
Hotel not found...skipping
Hotel not found...skipping
Hotel not found...skipping
Hotel not found...skipping
Hotel not found...skipping


In [12]:
# 7. Drop the rows where there is no Hotel Name.
hotel_df.dropna()
clean_hotel_df = hotel_df
clean_hotel_df.head()

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
18,Saint-Philippe,RE,83.75,broken clouds,-21.3585,55.7679,"Chambres d'hôte ""La Trinité"""
21,Vondrozo,MG,79.65,broken clouds,-22.8167,47.2833,
22,Vaini,TO,75.36,moderate rain,-21.2000,-175.2000,Keleti Beach Resort
28,Tabou,CI,78.85,light rain,4.4230,-7.3528,hotêl doufoulougou
32,Rikitea,PF,79.34,few clouds,-23.1203,-134.9692,People ThankYou


In [13]:
# 8a. Create the output File (CSV)
output_data_file = "Vacation_Search/WeatherPy_vacation.csv"
# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")


In [25]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current Weather</dt><dd>{Current Description} and {Max Temp} °F</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

In [26]:
# 11a. Add a marker layer for each city to the map. 
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(marker_layer)
# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))